In [8]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *
from fastai.text import *
from fastai.column_data import *

In [9]:
from fsm import *

In [10]:
DATA_PATH = Path('data')
DATA_PATH
TMP_PATH = DATA_PATH/'tmp'
TMP_PATH.mkdir(exist_ok=True)

In [11]:
sql_ids = np.load(TMP_PATH/'sql_ids.npy')
sql_itos = pickle.load(open(TMP_PATH/'sql_itos.pkl','rb'))
sql_stoi =  collections.defaultdict(lambda: 3, {v:k for k,v in enumerate(sql_itos)})

In [12]:
def get_next_random(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    p = to_np(p)[0]
    pi = p.argsort()[-10:]
    i = np.random.choice(pi, p=softmax_np(p[pi]))
    return indices_char[i]
def softmax_np (nparray):
    ans = np.exp(nparray)
    return ans/ans.sum()

def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    p = to_np(p)
    
    return indices_char[i]

def get_sentence(inp):
    sentence = inp
    counter = 0
    tmp = []
    while "_eos_" not in tmp:   
        tmp = get_next(sentence[-8:]) # always get last three
        sentence += [tmp]
        counter = counter+1
        if counter> 500:
            break
    return sentence
        

def get_sentence_random(inp):
    sentence = inp
    counter = 0
    tmp = []
    while "_eos_" not in tmp:   
        tmp = get_next_random(sentence[-8:]) # always get last three
        sentence += [tmp]
        counter = counter+1
        if counter> 500:
            break
    return sentence
        

def show_sentence(inp):
    sentence =''
    captialize = False
    for i in inp:
        if i =='t_up':
            captialize = True
        else:            
            if captialize:
                sentence = sentence + " " + i.upper()       
           
            else:
                sentence = sentence +" " + i
            captialize = False
    return sentence

In [46]:
# fsm = Rule_Parse_FSM()

# get_next_random("select userid from users where ".split())

# get_next_random_fsm("select userid from users where ".split(), fsm)

## Setup

Set up directory path

In [47]:
# dataset = pd.read_json(DATA_PATH/'train.json')
# dataset.head()


In [48]:
# get_next_random("select userid from users where ".split())

# SQL = dataset.sql

# ## Transform the text to be tokenize readily, as certain token has special meaning in SQL, ie. "." means the schema relation
# token_replaced = {',' : ' , ',
#                  '#' : ' # ',
#                  '@' : ' @ ',
#                  '!' : ' ! ',
#                  '.' : ' . ',
#                  '%' : ' % ',
#                  '?' : ' ? ',
#                  ')' : ' ) ',
#                  '(' : ' ( ',
#                  '=' : ' = ',
#                  '/' : ' / ',
#                  '*' : ' * '}
    

In [49]:
# def fixup(x):
#     for bereplaced, replace in token_replaced.items():        
#         x = x.replace(bereplaced, replace).lower() # CASE insentive for SQL
#     return x

In [50]:
# SQL = SQL.apply(fixup)## Sanity Check
# SQL.iloc[0]

# SQL.iloc[0]

# sql_tok = Tokenizer.proc_all_mp(partition_by_cores(SQL)) 

# sql_tok[0]

NameError: name 'sql_tok' is not defined

In [ ]:
# def toks2ids(tok,pre):
#     freq = Counter(p for o in tok for p in o)
#     itos = [o for o,c in freq.most_common()]
#     itos.insert(0, '_bos_')
#     itos.insert(1, '_pad_')
#     itos.insert(2, '_eos_')
#     itos.insert(3, '_unk')
#     stoi = collections.defaultdict(lambda: 3, {v:k for k,v in enumerate(itos)})
#     ids = np.array([([0] + [stoi[o] for o in p] + [2]) for p in tok]) # pad bos at beginning and eos at the end
#     np.save(TMP_PATH/f'{pre}_ids.npy', ids)
#     pickle.dump(itos, open(TMP_PATH/f'{pre}_itos.pkl', 'wb'))
#     return ids,itos,stoi

In [21]:
# sql_ids,sql_itos,sql_stoi = toks2ids(sql_tok,'sql')

In [13]:
sql_ids_concat= []
for i in sql_ids:
    sql_ids_concat += i

In [14]:
sql_ids.shape

(24594,)

In [15]:
vocab_size = len(sql_itos)
print('total chars:', vocab_size)

total chars: 23125


### Create inputs

Create a list of every 4th character, starting at the 0th, 1st, 2nd, then 3rd characters

In [16]:
idx = sql_ids_concat

In [17]:
char_indices = sql_stoi
indices_char = sql_itos

### Create and train model

Pick a size for our hidden state

In [18]:
n_hidden = 256
n_fac = 42

## Multi-output model

In [19]:
cs=5

In [20]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(0, len(idx)-cs-1, cs)]

In [21]:
c_out_dat = [[idx[i+j] for i in range(cs)] for j in range(1, len(idx)-cs, cs)]

In [22]:
xs = np.stack(c_in_dat)
xs.shape

(429356, 5)

In [23]:
ys = np.stack(c_out_dat)
ys.shape

(429356, 5)

In [24]:
xs[:3,:cs]

array([[  0,  13,  18,   4, 102],
       [  5,  18,   4,  33,   5],
       [ 18,   4,  91,   5,  18]])

In [25]:
ys[:3]

array([[ 13,  18,   4, 102,   5],
       [ 18,   4,  33,   5,  18],
       [  4,  91,   5,  18,   4]])

### Create and train model

In [34]:
val_idx = get_cv_idxs(len(xs)-cs-1)

In [35]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, ys, bs=64)

## Stateful model

### GRU

In [36]:
import pdb

In [37]:
def nll_loss_seq(inp, targ):
    sl,bs,nh = inp.size()
    targ = targ.transpose(0,1).contiguous().view(-1)
    return F.nll_loss(inp.view(-1,nh), targ)

In [38]:
class CharSeqStatefulGRU(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn1 = nn.GRU(n_fac, n_hidden)
        self.rnn_drop1 =nn.Dropout(p=0.5,inplace=True)
        self.rnn2 = nn.GRU(n_fac, n_hidden)
        self.rnn_drop2 =nn.Dropout(p=0.5,inplace=True)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn1(inp, self.h)
        self.h = repackage_var(h)
#         pdb.set_trace()
        return F.log_softmax(self.l_out(outp), dim=-1)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

* cs is the input , cs = 5 for sequence Length, a tuple with length(cs)
* each tensor is 512, length = bs # batch size
* `(Pdb) torch.stack(cs).size()`
* `torch.Size([5, 512])`
* `(Pdb) outp.size()
torch.Size([5, 512, 256])`
*`(Pdb) !h.size()
torch.Size([1, 512, 256])`
*(Pdb) self.l_out(outp).size()
torch.Size([5, 512, 23125])
*(Pdb) self.l_out
Linear(in_features=256, out_features=23125, bias=True)



In [39]:
# m.rnn.weight_hh_l0.data.copy_(torch.eye(n_hidden))

In [40]:
m = CharSeqStatefulGRU(len(sql_itos), n_fac, 512)
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))
learn = RNN_Learner(md, SingleModel(to_gpu(m)), opt_fn=opt_fn)

In [44]:
learn.lr_find()

NameError: name 'learn' is not defined

In [45]:
learn.sched.plot()

NameError: name 'learn' is not defined

In [41]:
fit(m, md, 1, opt, nll_loss_seq)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

  0%|          | 17/5367 [00:08<42:16,  2.11it/s, loss=8.98]


KeyboardInterrupt: 

In [29]:
n
n
fit(m, md, 6, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.68409  1.67784]                                 
[ 1.       1.49813  1.52661]                                 
[ 2.       1.41674  1.46769]                                 
[ 3.       1.36359  1.43818]                                 
[ 4.       1.33223  1.41777]                                 
[ 5.       1.30217  1.40511]                                 



In [30]:
set_lrs(opt, 1e-4)

In [31]:
fit(m, md, 3, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.22708  1.36926]                                 
[ 1.       1.21948  1.3696 ]                                 
[ 2.       1.22541  1.36969]                                 



In [18]:
os.makedirs(f'{PATH}models', exist_ok=True)

In [19]:
fit(m, md, 2, lo.opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.72032  1.64016]                                 
[ 1.       1.62891  1.58176]                                 



## Test Sentence

In [ ]:
def get_next_random_fsm(inp, fsm):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    p = to_np(p)[0]
    pi = p.argsort()[-10:]
    i = np.random.choice(pi, p=softmax_np(p[pi]))
    
    if fsm.run(indices_char[i]):
        return indice_char[i]
    else: 
        for pii in pi:
            if fsm.run(indice_char[pii]):
                print('pii loop')
                return indice_char[pii]
            else:
                print('Error, no valid words at all')  
    
    
    return "ERROR"

fsm = Rule_Parse_FSM()

get_next_random("select userid from users where ".split())

get_next_random_fsm("select userid from users where ".split(), fsm)

In [ ]:
a=get_sentence(['select','*','from','where','users'])

b=get_sentence_random(['select','*','from','where','users'])

show_sentence(b)

show_sentence(a)

show_sentence(a)

get_next(['select', '*', 'from'])